In [ ]:
from ipywidgets import interact_manual, interactive, GridspecLayout, AppLayout, Button, Layout, Widget
from IPython.display import display
import matplotlib
import pandas as pd
import ipywidgets as widgets
import io
import plotly.express as px
import plotly.io as pio
pio.templates.default = "simple_white"

In [ ]:
# DATA FOR STRESS COMPARISONS
# retreive files with ANOVA data
a0 = pd.read_excel("results_part1.xlsx")
b0 = pd.read_excel("results_part2.xlsx")
c0 = pd.read_excel("results_part3.xlsx")
#d0 = pd.read_csv("results_part4.csv")

# combine them
result0 = pd.concat([a0, b0, c0])
result0.dropna(subset=["Gene Symbol"], inplace=True)
result = result0.groupby(['Gene Symbol', 'Tissue', 'Comparison', "Experiment"])[
    ['p-value', 'Fold-change']].agg([('mean')]).reset_index()
result.columns = result.columns.get_level_values(0)


# load the expression data and combine/ fix older ones
a = pd.read_csv("Expressions_all_part1.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
b = pd.read_csv("Expressions_all_part2.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
c = pd.read_csv("Expressions_all_part3.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
d = pd.read_csv("Expressions_all_part4.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
e = pd.read_csv("Expressions_all_part5.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
f = pd.read_csv("Expressions_all_part6.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
g = pd.read_csv("Expressions_all_part7.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
h = pd.read_csv("Expressions_all_part8.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
i = pd.read_csv("Expressions_all_part9.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
j = pd.read_csv("Expressions_all_part10.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
k = pd.read_csv("Expressions_all_part11.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
l = pd.read_csv("Expressions_all_part12.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
m = pd.read_csv("Expressions_all_part13.csv", dtype={
                'Gene Symbol': object, 'Experiment': object})
# combine them
expressions = pd.concat([a, b, c, d, e, f, g, h, i, j,
                         k, l, m], sort=False).reset_index()
# clean up a bit
expressions.drop(expressions.columns[0], axis=1, inplace=True)
expressions.drop(expressions.columns[0], axis=1, inplace=True)

In [ ]:
sexes = []
expressions["Sex"]=''
for i in expressions["Mouse"]:
    if "_F_" in i:
        sexes.append("Female")
    if "_M_" in i:
        sexes.append("Male")
    if "_M_" not in i:
        if "_F_" not in i:
            sexes.append("Male")
expressions["Sex"]=sexes



expressions["Genotype "]=''
wts=[]
for i in expressions["Mouse"]:
    if "PKO" in i:
        wts.append("Pacap KO")
        
    if "KONS" in i:
        wts.append("Pacap KO")
    if "KOST" in i:
        wts.append("Pacap KO")
        
    if "M_KO" in i:
        wts.append("Pacap KO")
    if "F_KO" in i:
        wts.append("Pacap KO")
    if "PACAP_KO" in i:
        wts.append("Pacap KO")
    if "PAC1_KO" in i:
        wts.append("Pac1 KO")
    if "PAC1_WT" in i:
        wts.append("Wildtype")
    if "PACAP_WT" in i:
        wts.append("Wildtype")
    if "PWT" in i:
        wts.append("Wildtype")
    if "M_WT" in i:
        wts.append("Wildtype")
    if "F_WT" in i:
        wts.append("Wildtype")
    if "WTST" in i:
        wts.append("Wildtype")
    if "WTNS" in i:
        wts.append("Wildtype")

expressions["Genotype "] = wts



This program allows exploration of the Eiden Lab microarray data sets 2019.03, 2019.04, and 2019.12. Press the double arrow above and then confirm "restart and run all cells".

Press the Reset button to Reset after an analysis)

NOTE: As of 8/31/2020 this program does not contain ANOVA comparisons between male and female animals or between Pooled Wildtype animals and other groups in 2019.03 hypothalamus experiment. 

In [ ]:
from traitlets import traitlets
# Create a class of buttons which allows me to check which one was pressed


class LoadedButton(widgets.Button):
    """A button that can hold a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))


button_layout = widgets.Layout(
    border='solid 1px black', height='auto', width='auto')


def create_expanded_button(description, button_style):
    return LoadedButton(description=description, button_style=button_style, layout=button_layout)

In [ ]:
# define GUI
def choose_gene(change):
    if dropdown.value not in genes2:
        if dropdown.value != '':
            genes2.append(dropdown.value)
            with outgenes:
                display(dropdown.value)


graph_out1 = widgets.Output()
graph_out2 = widgets.Output()


def on_value_change(change):
    # just be here
    print('')


def get_vals(go_button):
    a = result[result["Comparison"].isin(list(comps.value))]
    stats = a[a["Gene Symbol"].isin(genes2)]

    sig_genes0 = a["Gene Symbol"].tolist()
    sig_genes = list(set(sig_genes0))
# Organize the expression values to include selected data
    mouse_means0 = expressions[expressions["Gene Symbol"].isin(genes2)]
    a = list(comps.value)
    b = list(set(expressions["Geno and Condition"]))
    com = []
    for geno in b:
        for comp in a:
            if geno in comp:
                com.append(geno)
    mice_to_include = list(set(com))
    mouse_means1 = mouse_means0[mouse_means0["Geno and Condition"].isin(
        mice_to_include)]


    div_list =['Gene Symbol']
    if tissue.value == True:
        div_list.append('Tissue')
    if experiment.value == True:
        div_list.append('Experiment')
    if sex.value == True:
        div_list.append('Sex')
    if genocond.value == True:
        div_list.append('Geno and Condition')
    if genotype.value == True:
        div_list.append('Genotype')
    if stress.value == True:
        div_list.append('Condition')
        
    if 'Genotype' in div_list:
        if comb_wt.value == True:
            div_list.remove("Genotype")
            div_list.append("Genotype ")
        if 'Condition' in div_list:
            if 'Geno and Condition' not in div_list:
                if len(div_list)>5:
                    div_list.remove("Genotype")
                    div_list.remove("Condition")
                    div_list.append('Geno and Condition')##if both geno and condition in divlist, group by combo column instead


    mouse_means2 = (mouse_means1.groupby(div_list)['Expression']
                        .agg([('average', 'mean'), ('error', 'sem')])
                        .reset_index())

    v = list(mouse_means2.columns)


    return(mouse_means2, stats, sig_genes, mice_to_include, v, div_list)

In [ ]:
# Graph it
def on_go_button_clicked(go_button):
    mouse_means2, stats, sig_genes, mice_to_include, v, div_list = get_vals(go_button)
# Mark errors
    if len(genes2) < 1:
        with outerrors:
            print("Choose at least one Gene")
    else:
        # print(mouse_means2)
       # Graph expression
        import plotly.graph_objects as go
        with graph_out1:
            # Graph that
            fig = px.bar(
                mouse_means2, x=v[1],
                y="average",
                facet_col="Gene Symbol",
                color=v[2],
                barmode='group',
                color_discrete_sequence=px.colors.sequential.Plasma,
                error_y="error"
            ).for_each_trace(lambda t: t.update(name=t.name.replace("=", ": ")))
            for i, a in enumerate(fig['layout']['annotations']):
                if a['text'][:5] == 'Exper':
                    a['text'] = ''
                if a['text'][:5] == 'Gene ':
                    a['text'] = a['text'].split("=")[1]
            fig.update_layout(
                title="Expression levels of selected genes",
                yaxis_title="Expression",
                xaxis={'categoryorder': 'total descending'})
            fig.update_xaxes(automargin=True, tickangle=45)
            fig.show()

       # Graph statistics
        with graph_out2:
            fig2 = px.bar(stats, x="Comparison", y="Fold-change",
                          # width=800, height=400,
                          facet_col="Gene Symbol",
                          facet_row="Experiment",
                          color_discrete_sequence=px.colors.sequential.Plasma,
                          # color = "p-value", range_color=[0,p_threshold.value], #Change so user input tells it what p and fc to use
                          color="Experiment",
                          hover_name="Tissue",
                          hover_data=["Experiment", "Fold-change", "p-value"]
                          ).for_each_trace(lambda t: t.update(name=t.name.replace("=", ": ")))
            fig2.update_yaxes(title_text='')
            fig2.update_xaxes(title_text='', tickangle=45)
            for i, a in enumerate(fig2['layout']['annotations']):
                if a['text'][:5] == 'Exper':
                    a['text'] = ''
                if a['text'][:5] == 'Gene ':
                    a['text'] = a['text'].split("=")[1]
            fig2.update_layout(title="ANOVA results",
                               margin=dict(

                                   b=250))
            # fig.update_xaxes(automargin=True)
            fig2.show()


In [ ]:
allgenes = list(set(result["Gene Symbol"]))
style = {'description_width': 'initial'}
genes2 = []
spacer = widgets.Label('')
genetitle = widgets.Label('Select Genes')

dropdown = widgets.Combobox(
    options=[''],
    description='Gene',
    placeholder='Type here',
    layout={'width': '80%'})
dropdown.options = allgenes

outgenes = widgets.Output()
outgraph = widgets.Output(layout={'border': '1px solid black'})


gene_button = widgets.Button(description='Enter Gene',
                             button_style='',
                             display='flex',
                             flex_flow='column',
                             align_items='stretch')
gene_button.on_click(choose_gene)

go_button = widgets.Button(description='Produce Output',
                           button_style='',
                           display='flex',
                           flex_flow='column',
                           align_items='stretch')
go_button.on_click(on_go_button_clicked)


gene_button.observe(dropdown.value)
go_button.observe(dropdown.value)


graph_out1 = widgets.Output()
graph_out2 = widgets.Output()

In [ ]:
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    border= '1px solid black',
                    width='100%',
                    )

box_layout2 = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border= '1px solid black',
                    width='100%',
                    )

In [ ]:
dividedby0 = widgets.Label('CHOOSE TWO:')
dividedby = widgets.Label('Group data by:')
sex = widgets.Checkbox(
    value=False,
    description='Sex',
    disabled=False,
    indent=False,
    layout=Layout(width='15%', height='12px'),
)
genotype = widgets.Checkbox(
    value=False,
    description='Genotype',
    disabled=False,
    indent=False
)
stress = widgets.Checkbox(
    value=False,
    description='Stress',
    disabled=False,
    indent=False,
    layout=Layout(width='15%', height='12px')
)
tissue = widgets.Checkbox(
    value=False,
    description='Tissue',
    disabled=False,
    indent=False
)
experiment = widgets.Checkbox(
    value=False,
    description='Experiment',
    disabled=False,
    indent=False,
    layout=Layout(width='15%', height='12px')
)
genocond = widgets.Checkbox(
    value=False,
    description='Genotype and Stress',
    disabled=False,
    indent=False
)
comb_wt = widgets.Checkbox(
    value=False,
    description='Combine PACAP and PAC1 WT?',
    disabled=False,
    indent=False
)

checksA = widgets.HBox([sex, genotype])
checksB = widgets.HBox([stress, tissue])
checksC = widgets.HBox([experiment, genocond])
checks = widgets.VBox([dividedby0,dividedby,checksA, checksB, checksC, comb_wt], layout = box_layout2)


##Need to make it grey out after choosing two 

In [ ]:
style = {'description_width': '200px'}

opts = ['Pacap KO Non-stressed vs. Pacap WT Non-stressed', 'Pacap KO Stressed vs. Pacap KO Non-stressed',
        'Pacap KO Stressed vs. Pacap WT Non-stressed', 'Pacap KO Stressed vs. Pacap WT Stressed',
        'Pacap WT Stressed vs. Pacap WT Non-stressed', 'Pac1 KO Non-stressed vs. Pac1 WT Non-stressed',
        'Pac1 KO Non-stressed vs. Pacap KO Non-stressed', 'Pac1 KO Stressed vs. Pac1 KO Non-stressed', 'Pac1 KO Stressed vs. Pac1 WT Stressed',
        'Pac1 WT Non-stressed vs. Pacap WT Non-stressed', 'Pac1 WT Stressed vs. Pac1 WT Non-stressed', 'Pac1 WT Stressed vs. Pacap WT Stressed']
comps = widgets.SelectMultiple(
    options=opts,
    layout=box_layout2,
    style = style,
    description='Include the following comparisons: ',
    disabled=False
)


In [ ]:

genes_out_title = widgets.Label('Genes Chosen:')
from IPython.core.display import display, HTML
s = HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells_above()"><input type="submit" id="toggleButton" value="Reset"></form>''')

genesbox = widgets.VBox([genetitle, dropdown])
genes_out = widgets.VBox([spacer, genes_out_title, outgenes])
buttons = widgets.HBox([gene_button, go_button])
genechoices = widgets.HBox([genesbox, genes_out])

outerrors = widgets.Output()
graphit = widgets.VBox([graph_out1, graph_out2])
# Put seletcors for p and fc values here
top_app = widgets.VBox([genechoices, outerrors, buttons], layout=box_layout2)


app = widgets.VBox([checks,comps,top_app, graphit], layout=box_layout)

app

In [ ]:
s